## Region-Based Segmentation

# PART 3

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow

# access google drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/Colab Notebooks/MSFD.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/') # Extracts to the current working directory (/content/)


### Otsu thresholding

In [ ]:
import cv2
import numpy as np
import os
import random

def calculate_IoU(original_image, thresholded_image):
    # Compute IoU
    # Reshape to the shape fo the thresholded_image
    original_image = cv2.resize(original_image, (thresholded_image.shape[1], thresholded_image.shape[0]))
    intersection = np.logical_and(original_image, thresholded_image)
    union = np.logical_or(original_image, thresholded_image)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def calculate_Dice(original_image, thresholded_image):
    # calculate dice score
    # Reshape to the shape fo the thresholded_image
    original_image = cv2.resize(original_image, (thresholded_image.shape[1], thresholded_image.shape[0]))
    intersection = np.logical_and(original_image, thresholded_image)
    dice_score = 2.0 * np.sum(intersection) / (np.sum(original_image) + np.sum(thresholded_image))
    return dice_score

train_folder = '/content/MSFD/1/face_crop'
test_folder = '/content/MSFD/1/face_crop_segmentation'

train_images = os.listdir(train_folder)
test_images = os.listdir(test_folder)

# Create sets for quick lookup
test_image_set = set(test_images)

total_iou = 0
total_dice = 0
image_count = 0
unmatched_count = 0

for train_filename in train_images:
    if train_filename in test_image_set:
        image_path = os.path.join(train_folder, train_filename)
        gt_path = os.path.join(test_folder, train_filename)

        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

        if image is not None and gt is not None:
            # Apply Otsu's thresholding
            _, thresholded_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            iou = calculate_IoU(gt, thresholded_image)
            total_iou += iou
            dice = calculate_Dice(gt, thresholded_image)
            total_dice += dice
            image_count += 1
    else:
        unmatched_count += 1

if image_count > 0:
    average_iou = total_iou / image_count
    average_dice = total_dice / image_count
    print("IoU for Otsu thresholding:", average_iou)
    print("Dice for Otsu thresholding:", average_dice)
    print("Unmatched images:", unmatched_count)
else:
    print("No matching images found in the specified folders.")
